In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import csv
import copy
import sklearn.linear_model
import json
import re
from sklearn import preprocessing
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
COLOR_TREAT = "#2ecc71"
COLOR_NO_TREAT = "#e74c3c"
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def percent_categorical(item, df=None, grouper='Active Status' , order = None , title = None , label = None , legend = None) :
    # plot categorical responses to an item ('column name')
    # by percent by group ('diff column name w categorical data')
    # select a data frame (default is IA)
    # 'Active Status' is default grouper

    # create df of item grouped by status
    grouped = (df.groupby(grouper)[item]
    # convert to percentage by group rather than total count
                .value_counts(normalize=True)
                # rename column 
                .rename('percentage')
                # multiple by 100 for easier interpretation
                .mul(100)
                # change order from value to name
                .reset_index()
                .sort_values(item))

    # create plot
    
    f, axarr = plt.subplots(1, 2, figsize = (15,8))
    
    sns.barplot(x=item,
                         y='percentage',
                         hue=grouper,
                         data=grouped,
                         palette='RdBu' , order = order , ax = axarr[0]
                         )
    
    
    
    sns.countplot( x = item , data= df , hue = grouper , palette='RdBu' , order = order , ax = axarr[1])
    
    
    
    
    if title == None:
        title = ['' , '']
    
    axarr[0].set_title(title[0] , size =15)
    axarr[1].set_title(title[1] , size =15)
    if label == None :
        label = item
    axarr[0].set_xlabel(label , size = 15)
    axarr[1].set_xlabel(label , size =15)
    
    axarr[0].tick_params(axis='both', which='major', labelsize=15)
    axarr[1].tick_params(axis='both', which='major', labelsize=15)
    
    axarr[0].set_xticklabels(order , rotation=45 )
    axarr[1].set_xticklabels(order , rotation=45 )
    
    axarr[0].set_ylabel('Percent' , size = 15)
    axarr[1].set_ylabel('Counts',  size =15 )
    
    if legend != None:
        
        for ax in axarr :
            leg_handles = ax.get_legend_handles_labels()[0]
            ax.legend(leg_handles, [legend[1], legend[2]], title=legend[0])
            plt.setp(ax.get_legend().get_texts(), fontsize='15') # for legend text
            plt.setp(ax.get_legend().get_title(), fontsize='15')
    
    return axarr

# Data exploration of conflicts and agreements
## Naïve analysis of the peace agreement dataset :

The Peace agreements dataset contains a large set of variable (69) defining the type of the agreement signed. We list below a few explanations of the features.

** PAID : ** Peace Agreement ID

**Region : **
The regional variable specifies the regional location of the conflict:
1. Europe: Geographic definition, including the states in the Caucasus. (COW numbers 200–395) 
2. Middle East: Egypt, Iran, Iraq, Israel, Jordan, Kuwait, Lebanon, Syria, Turkey, and the states of the Arabian Peninsula (COW numbers 630–698)
3. Asia: Geographic definition, including Oceania, Australia, and New Zealand, and excluding states in the Middle East. (COW numbers 700–990)
4. Africa: Geographic definition, excluding states in the Middle East (eg. Egypt). (COW numbers 400– 625)
5. Americas: Geographic definition, including states in the Caribbean. (COW numbers 2–165)

**GWNO : ** Gleditsch & Ward System Membership Table, it represents the country code

**CID : ** - Conflict ID

** Ended : ** 1 represents a PA that was terminated, 0 its success.

This dataset contains also some binary features to define the type of the agreement. There are binary features for :
- Behavioral conduct of the warring parties
- Regulation of governmental incompatibility
- Regulation of territorial incompatibility
- Justice issues
- Implementation issues
- Termination Variables : which corresponds to the evaluation of the agreement
- Type of process : is a full or partial peace agreement process

In [ ]:
pa = pd.read_excel('data/ucdp-peace-agreements.xls')
cols = ['bwdID', 'pa_comment', 'c_sign', 'c_3rd', 'c_duration', 'Reaffirm ID', 'Comment', 'txt', 
        'Link to fulltext agreement', 'ProcID', 'Frame', 'All', 'Counter' , 'Duration']
pa.drop(columns=cols, inplace=True)
regiondic = {
    1:'Europe',
    2:'Middle East',
    3:'Asia',
    4:'Africa',
    5:'Americas'
}
pa['Region'].replace(regiondic , inplace = True)

searchfor = ['UN', 'United Nations' , 'united nations' , 'ONU']
pa_UN = pa[pa['pa_3rd'].astype(str).str.contains('|'.join(searchfor))]
inter = list(set(pa_UN.PAID).intersection(pa.PAID))
pa["UN"] = ['UN intervention' if ele in inter else 'No UN intervention' for ele in pa.PAID]

## Distribution of the agreements by region

In [ ]:
sns.countplot(x='Region', data=pa, color=COLOR_TREAT, order=pa.Region.value_counts().index)
plt.title('Agreement counts per region')
plt.xlabel('Region');

This plot shows the number of peace agreements per region. We can see that most agreements concern African conflicts. It should be noted that this plot sums the number of Peace agreements per region. There are more peace agreements signed in the african region but the percentage of conflicts that are treated by Peace agreements differs a lot.

## Distribution of agreements by year

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot(x='Year', data=pa, color=COLOR_TREAT)

In [ ]:
plt.figure(figsize=(15,5));
sns.countplot(x='Year', data=pa, hue='UN', palette='RdBu');
plt.title('Number of the Peace Agreements by year', Fontsize=18)
plt.legend( ['Without' , 'With'] ,fontsize = 'large' , title = 'UN intervention')

## Year distibution of Peace agreements
1991 is the year where most peace agreements were signed. There has been two periods with a low amount of peace agreement, from 1983 to 1987 and from 2009 to 2011. Until 1990, The UN were not taking part to Peace agreements except for the Peace Process agreement between Namibia and South Africa in 1978.

In [ ]:
ax = sns.factorplot(kind='count', x='Inc', hue='UN', col='ended', size=6, data=pa, palette='RdBu');
plt.xlabel('Conflict incompatibility', fontsize=18)
ax.axes.flat[0].set_xlabel('Conflict incompatibility', fontsize=18)
ax.axes.flat[0].set_ylabel('Number of PAs', fontsize=18)
ax.axes.flat[0].set_title('Teminated PAs per conflict incompatibility', fontsize=17)
ax.axes.flat[1].set_title('Successful PAs per conflict incompatibility', fontsize=17)
plt.savefig('frontend/img/images/PA_Inc.pdf', transparent=True, bbox_inches='tight')

In [ ]:
percent_categorical('pa_type' , df=pa , grouper = 'UN' , order = pa.pa_type.value_counts().index,
                   title = ['Distribution of the PAs by type' , 'Number of PAs of each type'],
                   label = 'Type of agreement' , 
                   legend = ['UN intervention' , 'without' , 'with']);

## Executive strength of peace agreements
Most peace agreements are partial peace agreements. Peace Process agreements are generally agreements that are signed at the begginning of a conflict and call for future peace agreements. This figure that UN involvment in the peace agreements does not change much the distribution of the the PAs types.

## UN involvment in the Peace agrements
First create a new dataset with only the agreements where the UN took part then, create a new column in the pa dataset (1 if the agreement was signed by the UN, else).

In [ ]:
searchfor = ['UN', 'United Nations' , 'united nations' , 'ONU']
#pa_UN = pa[pa['pa_3rd'].astype(str).str.contains('UN')]
pa_UN = pa[pa['pa_3rd'].astype(str).str.contains('|'.join(searchfor))]
inter = list(set(pa_UN.PAID).intersection(pa.PAID))
pa["UN"] = [1 if ele in inter else 0 for ele in pa.PAID]

In [ ]:
#Let's check how many conflicts did the UN solved
pa.groupby("UN").PAID.count()

In [ ]:
to_be_categorical = ['termdur' , 'noconf11' , 'CoVi01' ,'DyVi05' ]
for categ in to_be_categorical:
    pa[categ] = pd.Categorical(pa[categ] , pa[categ].unique())

In [ ]:
plt.figure(figsize=(20,10))
sns.countplot(x='CID', data= pa_UN, hue='ended', order=pa_UN.CID.value_counts().index, palette='RdBu')

Out of the 216 agreements, The United Nations were involved as a third party in 68 of them which means 31%. We detect three groups of conflicts out of 35 concerned. 10 conflicts were not solved with the agreements, 3 conflicts were solved in three agrements with the one or two agreements that terminated. 11 conflicts had all of their respective agreements still valid as of 2011.


## Analysis of the Conflict ID variable

In [ ]:
print('Number of conflicts treated in this dataset : %s' %(len(pa.CID.unique())))

In [ ]:
plt.figure(figsize=(20,10))
sns.countplot(x='CID', data=pa, order=pa.CID.value_counts().index, color=COLOR_TREAT)

This plot shows that approximtely half of the conflicts were treated with one agreement. We wil split the rest of the exploration into two groups, the ones solved in one agreements and one grouping all the others.

In [ ]:
plt.figure(figsize=(20,10))
sns.countplot(x='CID', data= pa, hue='ended', order=pa.CID.value_counts().index, palette='RdBu')

Most conflicts ended with agreements that are still valid today. 
All agreements of the conflicts (179, 128, 177, 101, 216 205, 185) were terminated which means that none of them worked. 


## Analysis of the conflicts that could not be ended by the peace agreements

In [ ]:
pa_128 = pa.loc[pa['CID'] == 128]
#pa_128.to_csv('Unsolved_CID128.csv')

**Conflict 128 :**

Conflict between Iran and Iraq. Each of the 12 agreements were signed in 1975. Despite the first agreements, that was signed by The Shah of Iran and Saddam Hussein, all of them were signed by the same two representatives of Iran and Iraq, respectively by ABBAS-ALI KHALATBARY, Minister for Foreign Affairs of Iran and SAADOUN HAMADI, Minister for Foreign Affairs of Iraq. 
These agreements that were active in 1975, were than terminated between 1975 and 1980 until they were reactivated. Although the agreements were reactivated in 1980. Conflicts between Iran and Iraq did not stop in 1980, hence we can consider those agrrements as failures.

In [ ]:
pa_179 = pa.loc[pa['CID'] == 179]
#pa_179.to_csv('Unsolved_CID179.csv')

**Conflict 179 :**

This conflict concerned the Government of the Republic of Rwanda and the Rwandese Patriotic Front. The six agreements were signed between 1992 and 1993. The incompatibility is governmental.
The five first agreements were signed to provide partial peace and the last one in august 1993 was supposed to prvide full peace to the region. None of the agreements provided for the holding of national talks to solve incompatibility and for the integration of rebels into civil service.
In April 1994, a genocide broke out and the peace agreements were never fully implemented.

**Conflict 101 : **

This conflict originated from a governmental incompatibility between South Africa and Namibia. The agreement was signed in 1978 and lasted two months. The agreement contained provisions for political incompatibilities leading to independence for Namibia, justice provisions with the return of refugees and release of political prisoners and behavior regulations such as (ceasefire, withdrawal of foreign forces and demobilisation of local armies).
 
**Conflict 216 : **

Government of Guinea Bissau - Military Junta for the Consolidation of Democracy, Peace and Justice. Incompatibility 2, The peace agreement lasted for 6 months.

** Conflict 205 : **

The agreement covered indigenous rights and culture and strengthened the position of the indigenous people in Mexico. incompatibility 2. The accord was never implemented and thus further peace talks that were planned in September the same year were suspended by EZLN. No exact date for the ending of the peace accord has been found.

All of the conflicts that were treated by only one agreement and had terminated originated from governmental imcompabilities. 

## Analyze conflicts solved in one agreements that are still valid as of 2011


In [ ]:
idx = pa['CID'].value_counts()
idx_1 = idx[idx.values == 1]
idx_1.index

In [ ]:
indx_1_ended = []
for ind in idx_1.index:
    #print(ind)
    a = pa.ended[pa['CID'] == ind]
    if a.iloc[0] == 0: # ended is the column 17 
        indx_1_ended.append(a.index)
    
pa_one = pa.iloc[np.array(indx_1_ended)[:,0], :]
pa_one.head(2)

**Let's analyse the different combination of behavioral provisions that were taken and led to the success of the these agreements.**

In [ ]:
def whatcombi(pa):
    # returns the corresponding combination number of the dataset
    # set : n and rerun box in fct of the number of binary features to be analyzed
    n = 5
    # Create 2^n vectors of size n representing a combination
    lst = [list(i) for i in itertools.product([0, 1], repeat=n)]
    for i in range(len(lst)):
        a = pa == lst[i]
        # if all values in a are true, it is the good combination
        if all(x for x in a.values):
            return(i)
pa_1behavior = pa_one.iloc[:, 20:25]
pa_1behavior['combination'] = pa_1behavior.apply(whatcombi, axis=1)
sns.countplot(x='combination', data=pa_1behavior, order=pa_1behavior.combination.value_counts().index, color=COLOR_TREAT)

8 successfull agreements established peace in providing for a ceasefire. 3 provided for the withdrawal of foreign forces and 3 others provided for a ceasefire, the integration of rebels into the army and the disarmement of the warring parties.

**Let's analyse the different combination of governmental and territorial incompatibility provisions that were taken and led to the success of the these agreements.**

Because we merged the incompabilities, the number of possible combinations to analyse equals to $2^{18}$ hence we saved the results to save computation time.

In [ ]:
def whatcombi(pa):
    # returns the corresponding combination number of the dataset
    # set : n and rerun box in fct of the number of binary features to be analyzed
    n = 18
    # Create 2^n vectors of size n representing a combination
    lst = [list(i) for i in itertools.product([0, 1], repeat=n)]
    for i in range(len(lst)):
        a = pa == lst[i]
        # if all values in a are true, it is the good combination
        if all(x for x in a.values):
            return(i)
#pa_1incompatibility = pa.iloc[:, 25:43]
#pa_1incompatibility['combination'] = pa_1incompatibility.apply(whatcombi, axis=1)
#pa_1incompatibility.to_csv('pa_1PA_success.csv')
pa_1incompatibility = pd.read_csv('data/pa_1PA_success.csv')
#pa_1incompatibility['inc'] = pa.iloc[np.array(indx_1_ended)[:,0], 8]

In [ ]:
a = copy.deepcopy(pa_1incompatibility.groupby('combination').count().pp)
a.sort_values(ascending=False, inplace=True)

In [ ]:
plt.figure(figsize=(20,10))
a[:20].plot('bar', rot=45, color=COLOR_TREAT)
#sns.countplot(x='combination', data=pa_1incompatibility, order=pa_1incompatibility.combination.value_counts().index)

Most of the time, no incompatibility provisions is set in those successful agreements. The abscence of incompatibility provisions might be an insight to explain their success as they do not have high expectations. The second most recurrent agreements treat territorial incompatibilities and grant the disputed region local governance. Governmental incompatibilities are mostly solved with provisions for elections and an interim government.

## Whatcombi :  This function returns corresponding combination to a pandas row dataframe
This function is to be used for each type of provisions.

In [ ]:
def whatcombi(pa):
    # returns the corresponding combination number of the dataset
    # set : n and rerun box in fct of the number of binary features to be analyzed
    n = 2
    # Create 2^n vectors of size n representing a combination
    lst = [list(i) for i in itertools.product([0, 1], repeat=n)]
    for i in range(len(lst)):
        a = pa == lst[i]
        # if all values in a are true, it is the good combination
        if all(x for x in a.values):
            return(i)

## Analysis of the agreements termination consistency in the variables :

- **ended** 
    - 0) if the agreements still holds
    - 1) if the agreement terminated
    
- **DyVi05** 
    - 1) the dyad violence restarted or continued
    - 0) the violence ended in this dyad
    - -99) Not applicable, agreement signed previous year

In [ ]:
pa_result = pa.loc[:, ['ended', 'DyVi05']]
pa_result.head()

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
sns.countplot(x='DyVi05', data=pa, color=COLOR_TREAT)
plt.subplot(122)
sns.countplot(x='ended', data=pa, color=COLOR_TREAT)

Thoses plots show that approximately 140 agreements ended there corresponding conflict. This information should be crossed with the number of conflicts treated in this dataset as there are only 60 unique conflicts.

In [ ]:
pa_result['combination'] = pa_result.apply(whatcombi, axis=1)

In [ ]:
sns.countplot(x='combination', data=pa_result, color=COLOR_TREAT)

Approximately 160 out of 216 (74%) agreements have the same values for the *ended* and *DyVi05* variables. Hence we can say that those variables are consistent.

## Behaviour variables analysis

In [ ]:
pa_behavior = pa.iloc[:, 14:18]
pa_behavior.head()

In [ ]:
# run only once
def whatcombi(pa):
    # returns the corresponding combination number of the dataset
    # set : n and rerun box in fct of the number of binary features to be analyzed
    n = 4
    # Create 2^n vectors of size n representing a combination
    lst = [list(i) for i in itertools.product([0, 1], repeat=n)]
    for i in range(len(lst)):
        a = pa == lst[i]
        # if all values in a are true, it is the good combination
        if all(x for x in a.values):
            return(i)
pa_behavior['combination'] = pa_behavior.apply(whatcombi, axis=1)

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x='combination', data=pa_behavior, order=pa_behavior.combination.value_counts().index, 
              color=COLOR_TREAT)

1. Most agreements do not treat behavioral instructions
2. The second most recurrent combination concerns agreements that provided a ceasefire
3. third most recurrent :  ceasefire, Disarmement of foreign forces
4. fourth : ceasefire, integration in national army, disarmement of warring parties

In [ ]:
pa_behavior['ended'] = pa['ended']
pa_behavior['UN'] = pa['UN']
sns.factorplot(x='combination', data= pa_behavior, hue='ended', col='UN', kind='count', size=6, aspect = 1.2, 
               palette='RdBu', order=pa_behavior.combination.value_counts().index);

There are no much difference in agreements that terminate or not. However Two kinds of agreements were never terminated. The ones which provided for the creation of a new national army or the integration of rebels into the army and the withdrawal of foreign forces. No much difference can be established between UN agreements and the others considering the small amount of data available.

## Incompatibility variables analysis
### Government incompatibilities
Run the whatcombi box for n = 7 which leads to $2^7$ possible combinations

In [ ]:
def whatcombi(pa):
    # returns the corresponding combination number of the dataset
    # set : n and rerun box in fct of the number of binary features to be analyzed
    n = 7
    # Create 2^n vectors of size n representing a combination
    lst = [list(i) for i in itertools.product([0, 1], repeat=n)]
    for i in range(len(lst)):
        a = pa == lst[i]
        # if all values in a are true, it is the good combination
        if all(x for x in a.values):
            return(i)
pa_incompatibility_gov = pa.iloc[:, 19:26]
pa_incompatibility_gov.head()

In [ ]:
pa_incompatibility_gov['combination'] = pa_incompatibility_gov.apply(whatcombi, axis=1)

In [ ]:
pa_incompatibility_gov['UN'] = pa["UN"]
pa_incompatibility_gov['ended'] = pa['ended']
sns.factorplot(x='combination', data=pa_incompatibility_gov, hue='ended', col='UN', kind='count', size=6, aspect = 1.2,
              palette='RdBu', order=pa_incompatibility_gov.combination.value_counts().index)
plt.yscale('log')

While most agreements do not deal with governmental incompatibilities, the ones that do mostly provide for elections and electoral reforms for rebel integration into the interim government.

The main difference that can be observed between UN agreements and non UN agreements is that the UN agreements usually provide provisions for an interim government and elections while non UN agreements msotly only provide provisions for an election.

The *Pol_prov* feature could be used to generalize the inclusion of political provisions in the agreement

### Territorial incompatibilities
run whatcombi box with n = 9

In [ ]:
def whatcombi(pa):
    # returns the corresponding combination number of the dataset
    # set : n and rerun box in fct of the number of binary features to be analyzed
    n = 9
    # Create 2^n vectors of size n representing a combination
    lst = [list(i) for i in itertools.product([0, 1], repeat=n)]
    for i in range(len(lst)):
        a = pa == lst[i]
        # if all values in a are true, it is the good combination
        if all(x for x in a.values):
            return(i)
pa_incompatibility_ter = pa.iloc[:, 27:36]
pa_incompatibility_ter.head()

In [ ]:
pa_incompatibility_ter['combination'] = pa_incompatibility_ter.apply(whatcombi, axis=1)

In [ ]:
pa_incompatibility_ter['UN'] = pa["UN"]
pa_incompatibility_ter['ended'] = pa['ended']
plt.figure(figsize=(15,8))
sns.countplot(x='combination', data=pa_incompatibility_ter, hue='ended',
             palette='RdBu', order=pa_incompatibility_ter.combination.value_counts().index)
plt.yscale('log')

There is no significant difference between UN agreements and the rest to display it. However we can see with this plot that when agreements have territorial provisions, it generally concerns the demarcation of a border or the government granting the disputed region to the other warring party.

## Justice variables analysis
Run whatcombi box with n = 4

In [ ]:
def whatcombi(pa):
    # returns the corresponding combination number of the dataset
    # set : n and rerun box in fct of the number of binary features to be analyzed
    n = 4
    # Create 2^n vectors of size n representing a combination
    lst = [list(i) for i in itertools.product([0, 1], repeat=n)]
    for i in range(len(lst)):
        a = pa == lst[i]
        # if all values in a are true, it is the good combination
        if all(x for x in a.values):
            return(i)
pa_justice = pa.iloc[:, 37:41]
pa_justice.head()

In [ ]:
# run only once
pa_justice['combination'] = pa_justice.apply(whatcombi, axis=1)

In [ ]:
sns.countplot(x='combination', data=pa_justice, color=COLOR_TREAT);

In [ ]:
pa_justice['UN'] = pa["UN"]
pa_justice['ended'] = pa['ended']
sns.factorplot(x='combination', data=pa_justice, hue='ended', col='UN', kind='count', size=6, aspect = 1.2,
              palette='RdBu', order=pa_justice.combination.value_counts().index);

- The most recurring combination is the one where no justice issues is treated in the agreement.
- The second most recurring is the one where the agreement provided a release of prisoners

It seems like UN agreements uses a smaller amount of combinations than other agreements, particularly, the combination 13 corresponding to an amnesty, a release of prisoners and the return of refugees. Again this interpretation has to be understood with modesty as we lack data to be more consistent. A less naïve analysis will be made later to solve this issue.

The *Justice_prov* feature is set to one if any of the justice provisions was applied in the agreement. This variable could be of good use to reduce the number of features and keep an insight of the rather or not 'Justice' issues treated in the agreement.

## Implementation variables analysis

In [ ]:
pa_implementation = pa.iloc[:, 42:46]
# study ended results in fct of all combinations of the binary features
pa_implementation.head()

In [ ]:
# run only once
pa_implementation['combination'] = pa_implementation.apply(whatcombi, axis=1)

In [ ]:
sns.countplot(x='combination', data=pa_implementation, color=COLOR_TREAT);

In [ ]:
pa_implementation['UN'] = pa["UN"]
pa_implementation['ended'] = pa['ended']
sns.factorplot(x='combination', data=pa_implementation, hue='ended', col='UN', kind='count', size=6, aspect = 1.2,
              palette='RdBu', order=pa_implementation.combination.value_counts().index);

- The most recurring implementation is the one where the agreement did not reaffirm earlier agreements, did not outline a negotiating agenda including negotiations on the incompatibility, did not provide for the deployment of a peace-keeping operation and did not provide for the establishment of a commission or committee to oversee implementation of the agreement.

- The second most combination recurring only provided for the establishment of a commission or committee to oversee implementation of the agreement.

- The third most recurring combination outline a negotiating agenda including negotiations on the incompatibility

- Fourth : reaffirm earlier agreements and provided the establishment of a commission or committee to oversee implementation of the agreement.

A small diference between the agreements where the UN was involved and those where it wasn't are the ones where a provision for peace keeping operations is made. This is of course logical because the UN is the entity that generally decides for such missions.

## Temination variables
- **DyVi05** - Violence with the same parties restarted within 5 years

    - 1) Yes, the dyad violence restarted or continued
    - 0) No, the violence ended in this dyad
    - -99) Not applicable, agreement signed previous year


- **CoVi01** - Terminated the whole conflict the following year, signed an active year
    - 2) Part of a peace process which ended the violence.
    - 1) Yes, the conflict was terminated
    - 0) No, the conflict continued
    - -99) Not applicable, for example, the peace agreement was not signed in a conflict active year.


- **noconf** - Signed in a conflict inactive year.

The peace agreement was signed in an inactive conflict year i.e. in a year with less than 25 battle- related deaths recorded by the UCDP.
    - 1) Yes 
    - 0) No

- **termdur** - Number of years since last activity.

The number of inactive years between end of conflict and signature of the peace agreement.


- **Noconf11** - Still terminated as of 2011
    No active conflict years recorded from agreement until 2011 1) Yes, no active conflict years from agreement til 2011, 
    - 0) No, conflict restarted before 2011
    - -99) Not applicable, signed in 2011
    
The *termdur* will be used to relate agreement signed on active years to ones signed on inactive year. This should allow us to give more strentgh to an agreement that was signed fewer years after the end of the conflict than one signed a long time after. 

The *DyVi05* feature will be used as the second success evaluation variable. The *CoVi01* feature will be used to evaluate the time efficiency of the agreement.

# Check propensity scores

In [ ]:
#let's reimport the data as we used it a lot in the previous analysis, we want it to be clean
pa = pd.read_excel('data/ucdp-peace-agreements.xls')
cols = ['bwdID', 'pa_comment', 'c_sign', 'c_3rd', 'c_duration', 'Reaffirm ID', 'Comment', 'txt', 
        'Link to fulltext agreement', 'ProcID', 'Frame', 'All', 'Counter' , 'Duration']
pa.drop(columns=cols, inplace=True)

In [ ]:
#some further cleaning
#pa_UN = pa[pa['pa_3rd'].astype(str).str.contains('UN')]
#pa["UN"] = [1 if ele in pa_UN.CID else 0 for ele in pa.CID]

searchfor = ['UN', 'United Nations' , 'united nations' , 'ONU']
pa_UN = pa[pa['pa_3rd'].astype(str).str.contains('|'.join(searchfor))]
inter = list(set(pa_UN.PAID).intersection(pa.PAID))
pa["UN"] = [1 if ele in inter else 0 for ele in pa.PAID]
pa[pa['UN'] == 1]



to_be_categorical = ['termdur' , 'noconf11' , 'CoVi01' ,'DyVi05' ]
for categ in to_be_categorical:
    pa[categ] = pd.Categorical(pa[categ] , pa[categ].unique())
pa.groupby('UN').count()

In [ ]:
pa.iloc[:, 13:-2]

In [ ]:
import sklearn.linear_model
import sklearn.preprocessing
#removing the useless features
pa_reg = pa.iloc[:, 13:-2]

#prepocess by standardizing the dataset
pa_reg = preprocessing.scale(pa_reg)
#Perform a logistic regression in order to get the propensity score for each individuals
model = sklearn.linear_model.LogisticRegression()
model.fit(pa_reg, pa.UN)
pred = model.predict_proba(pa_reg)

Here we check the performance of our model, we have an accuracy of 89% which is quite good for a very simple logistic regression.

In [ ]:
#Check the performance of the model
sum(model.predict(pa_reg) == pa.UN)/len(pa_reg)

In [ ]:
#Scatter plot of the propensity score
plt.figure(figsize=(15, 10))
pa['pred'] = pred[:,1]
ax = sns.stripplot(x='PAID', y='pred', hue='UN', data=pa, palette={0:"#e74c3c", 1: "#2ecc71"})
ax.set(xticklabels=[], ylabel='Treatment (prediction)')
plt.title('Propensity score per agreement')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., handles=ax.get_legend_handles_labels()[0], labels=['Without', 'With'], title='UN')

This plot shows us what the propensity score for each PAID is. The UN PAID are in green and should have a score around 1 whereas the PAID without the UN are in red and should have a score around 0. Even if we have some outliers, this is what we observe on the plot.

In [ ]:
#Check that every PAID has a pred value
pa.pred.isnull().values.any()

Here we do the matching.

We saw in class that matching among the propensity score is almost the same as matching among the features.

In order to do this matching, we will procede in the following way:


- First, we will create a complete bipartite graph. It means that the vertices are partitioned into two subsets V1 (the people who received the training) and V2 (the people who did not receive the traing) such that no edge has both endpoints in the same subset, and every possible edge that could connect vertices in different subsets is part of the graph. This way will will be sure to match UN PAID people with PAID without the UN

- We will give a weight to these edges: Let's take two PAID,we denote by $w_1$ the propensity score of the UN PAID and $w_2$ the propensity score of the other PAID (no UN). The weight of the edge between them will be:
$$W = - |w_{1} - w_{2}| $$

- Finally we will use a maximum weight matching with max cardinality algorithm. Note that we have a minus sign in our weights because we want to minimize the weights even though we are using a maximum weight matching algorithm. We are using the method implemented in the NetworkX package. This method is based on the “blossom” method for finding augmenting paths and the “primal-dual” method for finding a matching of maximum weight, you can read more about it there : https://dl.acm.org/citation.cfm?id=6502 .

In [ ]:
import networkx as nx
from networkx.algorithms import bipartite

#create the nodes
G=nx.Graph()
G.add_nodes_from(pa['PAID'][pa.UN == 0])
G.add_nodes_from(pa['PAID'][pa.UN == 1])

In [ ]:
#Bipartite complete graph
for ID_u, score_u in zip(pa.PAID[pa.UN == 0], pa.pred[pa.UN == 0]):
    for ID_v, score_v in zip(pa.PAID[pa.UN == 1], pa.pred[pa.UN == 1]):
        G.add_edge(ID_u, ID_v, weight=-abs(score_u-score_v))

In [ ]:
#Let's check if the number of nodes and edges is right
#Number of nodes should be 185+429 = 614 (the graph is bipartite) and number of edges should be 185*429 = 79365
#because it is complete bipartite
print(G.number_of_nodes())
print(G.number_of_edges())

In [ ]:
#perform the matching
from networkx.algorithms import max_weight_matching
matching = max_weight_matching(G, maxcardinality=True)

In [ ]:
#We have 68 PAIDs that need to be matched, here we notice the dictionnary goes in both ways
len(list(matching.items()))

In [ ]:
pa_UN = pa[pa['UN'] == 1]
pa_UN['temp'] = 1
pa_UN = pa_UN[['PAID' , 'pred' , 'temp']]
pa_nUN = pa[pa['UN'] == 0]
pa_nUN['temp'] = 1
pa_nUN = pa_nUN[['PAID' , 'pred' , 'temp']]


#We merge the two datasets thanks to the temp column, this new merged dataset mimics the complete bipartite graph:
result = pd.merge(pa_nUN ,pa_UN , on='temp', how='left')
result = result[['PAID_x' , 'PAID_y' , 'pred_x' , 'pred_y']]
result['diff'] = abs(result['pred_x'] - result['pred_y'])
result = result.set_index(['PAID_x', 'PAID_y'])
result['diff'].isnull().values.any()
#result
print(result.loc[list(matching.items())].dropna().shape)
result.loc[list(matching.items())].dropna().head()

In [ ]:
paID = pa.set_index('PAID')
matched = paID.loc[list(matching.keys())]
plt.figure(figsize=(20,10))
sns.countplot(x='UN', data= matched, hue='ended')

The UN agreements are more 'ended' (so the conflicts start again) than the one without the UN intervention.
But note that this analysis is still biased! 
Indeed, we only looked at the peace agreements here, we also need to look at the conflicts from these peace agreements. Maybe the UN only solved very difficult conflicts.
In a second part we will look at the conflict that were solved and do another matching among them so we can redo this analysis with a filter on the conflict (will be done for markdown 3)

# Conflicts

## Cleaning the data

As we studied in the previous part, we will need to balance our study over the conflict that has be solved by the UN vs the others.

Indeed, the previous dataset only gave us informations about the peace agreement per se, we did not have any information about the conflict itself.

UCDP has another data set that describes the conflicts. It is named *ucdp-prio-acd*.

It is a conflict-year dataset with information on armed conflict where at least one party is the government of a state in the time period 1946-2016.

The Conflict IDs are not the same between *ucdp-peace-agreements* and *ucdp-prio-acd* because UCDP changed the way the give IDs. The dataset *ucdp-peace-agreements* uses the old ID notation while *ucdp-prio-acd* uses the new one . Therefore, we need the translation give bt UCDP, *translate_conf*, to link them.

In [ ]:
#Import the Peace Agreement dataset and the translation table to get the new IDs
pa = pd.read_excel('data/ucdp-peace-agreements.xls')
searchfor = ['UN', 'United Nations' , 'united nations' , 'ONU']
pa_UN = pa[pa['pa_3rd'].astype(str).str.contains('|'.join(searchfor))]
inter = list(set(pa_UN.PAID).intersection(pa.PAID))
pa["UN"] = [1 if ele in inter else 0 for ele in pa.PAID]
pa[pa['UN'] == 1]

reader = csv.reader(open('data/translate_conf.csv', 'r'))

#Create a dictionnary that will have {New ID : Old ID} so we can make the link
d = {}
for row in reader:
    k, v = row
    d[v] = k
    
#pa.CID = pa.CID.astype(dtype=str).replace(to_replace=d )
pa.CID = pa.CID.astype(str)
#print(type(pa.CID))
pa.CID = pa.CID.apply(lambda x: d[x])
pa.CID = pa.CID.astype(dtype = int)

In [ ]:
#Import the conflict dataset
df = pd.read_csv('data/ucdp-prio-acd-171.csv')

In [ ]:
#Let's observe the data
df.iloc[:10, :11]

In [ ]:
#Lot's of features so we do that in two times
df.iloc[:10, 11:27]

As we notice, there is several rows for the same conflict ID. Indeed, Each conflict is listed in the database and given a unique ID code. The temporal aspect of a conflict is not addressed by this definition; hence, two conflict episodes over the same incompatibility will be assigned the same ID regardless of the time separating them.

This is how UCDP records the data, as explained on their website:

- The observation (or unit) in the table is the conflict-year. Each conflict is listed in all years where fighting in one or more dyad(s) caused at least 25 battle-related deaths. The calendar year is the basic unit of every observation. Thus, if a conflict during the period June–September results in 30 casualties, that year will be recorded as a year of conflict. However, if the same number of casualties occurred in the period November–February and the conflict failed to reach the threshold of 25 battle-related deaths in either calendar year, neither year will be coded as in conflict. Start dates frequently refer to years prior to the first calendar year of a conflict, as the start of a conflict might be in a year with less than 25 fatalities. Small conflicts might not be included.

Taking this into account, we decide to refine this dataset in order to have a single conflict ID per row. In order to do that we will aggregate the dataset at the CID (Conflict ID) level. To do so we will drop some variables that only concern the years of observation and not the conflict in its totality. This variables are :

- `year` : The year of observation, which is not needed anymore as we will observe the conflict in its totality

- `intensity` : The intensity of the conflict in a given year.

- `startdate2` : The starting date of the observation (for a given year then, not the conflict itself).

- `startprec2` : The precision of the `startdate2` variable.

- `epend` : The ending date of the observation (same as above).

- `ependprec` : The precision of the `epend` variable.

These are the variables we droppend because we aggregate the dataset. We will drop some other variables because they are not needed in our analysis :

- `sidea2nd` : If there is a secondary actor in the conflict on side A, we don't think we need that much precision.

- `sideb2nd` : If there is a secondary actor in the conflict on side B, we don't think we need that much precision.

- `terr` : The disputed territory if there is any. We don't need that much precision for a propensity score.

- `startprec` : The precision of the starting date for the conflict. This is useless for our propensity score.

- `gwnoa`, `gwnoa2nd`, `gwnob`, `gwnob2nd`, `gwnoloc` : This is only country codes etc, we don't need that for a propensity score.

- `version` : The version of the dataset.

We will describe the variables we keep in the next markdown, after we show the clean data set.

In [ ]:
#Let's remove the features we don't need for our analysis
df.drop(['sidea2nd', 'sideb2nd' , 'terr' , 'year' , 'intensity' , 'startprec' , 'startdate2' , 'epend' , 'ependprec', 'gwnoa', 'gwnoa2nd', 'gwnob', 'gwnob2nd', 'gwnoloc', 'startprec2', 'version', 'sidebid'], axis=1, inplace = True)

In [ ]:
#Check how many conflicts we have
print('Number of unique conflicts in this dataset : %s' %(len(df.conflictid.unique())))

In [ ]:
print("Number of non-unique locations for a single conflict: %d" %len(df['conflictid'].unique()[(df.groupby(df['conflictid'])['location'].unique().apply(len) > 1)]))
print("Number of non-unique 'side a' for a single conflict: %d" %len(df['conflictid'].unique()[(df.groupby(df['conflictid'])['sidea'].unique().apply(len) > 1)]))
print("Number of non-unique 'side b' for a single conflict: %d" %len(df['conflictid'].unique()[(df.groupby(df['conflictid'])['side b'].unique().apply(len) > 1)]))
print("Number of non-unique incomp for a single conflict: %d" %len(df['conflictid'].unique()[(df.groupby(df['conflictid'])['incomp'].unique().apply(len) > 1)]))
print("Number of non-unique cumint for a single conflict: %d" %len(df['conflictid'].unique()[(df.groupby(df['conflictid'])['cumint'].unique().apply(len) > 1)]))
print("Number of non-unique types for a single conflict: %d" %len(df['conflictid'].unique()[(df.groupby(df['conflictid'])['type'].unique().apply(len) > 1)]))
print("Number of non-unique startdates for a single conflict: %d" %len(df['conflictid'].unique()[(df.groupby(df['conflictid'])['startdate'].unique().apply(len) > 1)]))
print("Number of non-unique regions for a single conflict: %d" %len(df['conflictid'].unique()[(df.groupby(df['conflictid'])['region'].unique().apply(len) > 1)]))

In [ ]:
#Let's aggregate the data to single CIDs (explanations in the markdown below)
clean_df = df.groupby('conflictid').last()
clean_df.head()

Here we will explain the variables we retained and how we aggregated them:

- `location` : The location of the conflict, this is always the same for each conflict ID, therefore we just take the value of the last observation for the aggregated dataset.

- `sidea` : The first agent in the conflict (it has to be a government), this is always the same for each conflict ID, therefore we just take the value of the last observation for the aggregated dataset.

- `side b` : The seconf agent in the conflict (it can be a government or an organisation), this is not always the same but after organisation we noticed that it was due to the organisation often changing name. Therefore we use the name of the organisation in the last observation for the aggregated dataset.

- `incomp` : The incompatibility that led to the conflict. this is always the same for each CID, therefore we simply take the value of the last observation. Here are the three possible values :
    - 1 : Territory
    - 2 : Government
    - 3 : Both
    
- `cumint` : It is a variable that says if the conflict did more than 1000 battle related deaths since its begining. Of course it is not the same accross the same CID because it is a cumulative variable, therefore we take the value for the last observation.

- `Type` : The type of conflict. The type can change for the same CID. Though, we think that the most important is what the conflict was at its end (we have the `incomp` variable to tell us how the conflict began). Therefore, we take the last value for the aggregated dataset. There are four types of conflicts :
    - 1 : Extrasystemic armed conflict occurs between a state and a non-state group outside its own territory.
    - 2 : Interstate armed conflict occurs between two or more states. 
    - 3 : Internal armed conflict occurs between the government of a state and one or more internal opposition group(s) without intervention from other states.
    - 4 : Internationalized internal armed conflict occurs between the government of a state and one or more internal opposition group(s) with intervention from other states (secondary parties) on one or both sides.
    
- `startdate` : The date of the start of the conflict, it is the same across all raws for the same CID so we can take whatever line when we aggregate.

- `ependdate` : The end date of the episode observed. If this is the last episode it is the date of the end of the conflict for the CID. We take the date of the last observation when we aggregate then.

- `region` : The region of the conflict, it is always the same across the same CID. Then we take the one for the last observation. The 5 regions are : 
    - 1 : Europe
    - 2 : Middle East
    - 3 : Asia
    - 4 : Africa
    - 5 : America
    
    
Now we will use the other dataset in order to know if there was a Peace Agreement for this conflict. We will add a 0/1 column to capture this information.

In [ ]:
#Check the CIDs that were ended by a peace agreement
agreements = pa.groupby('CID').last()
print('Number of unique conflicts that had a peace agreement : %s' %(len(agreements.index)))
print(agreements.index)

In [ ]:
#Check the CIDs that were ended by a peace agreement
UN_agreements = pa[pa['UN']==1].groupby('CID').last()
Other_agreements = pa[pa['UN']==0].groupby('CID').last()
print('Number of unique conflicts that had a UN peace agreement : %s' %(len(UN_agreements.index)))
print('Number of unique conflicts that had a peace agreement that did not involve the UN: %s' %(len(Other_agreements.index)))
#print(agreements.index)

In [ ]:
#We add a column so we know if the CID had a peace agreement (1) or not (0)
clean_df["agreement"] = [1 if ele in agreements.index else 0 for ele in clean_df.index]
clean_df["UN_agreement"] = [1 if ele in UN_agreements.index else 0 for ele in clean_df.index]
clean_df["Other_agreement"] = [1 if ele in Other_agreements.index else 0 for ele in clean_df.index]
clean_df.rename({'ependdate':'enddate'} , inplace = True, axis = 1)

In [ ]:
#Date time so we can substract and have the duration
clean_df['startdate'] = pd.to_datetime(clean_df['startdate'])
clean_df['enddate'] = pd.to_datetime(clean_df['enddate'])
clean_df['duration'] = (clean_df['enddate'] - clean_df['startdate']).dt.days
clean_df['duration'].fillna(0 , inplace = True)


#Categorical

to_be_categorical = ['incomp' , 'cumint' , 'type' ,'region' , 'agreement' , 'UN_agreement' , 'Other_agreement']
for categ in to_be_categorical:
    clean_df[categ] = pd.Categorical(clean_df[categ] , clean_df[categ].unique())

In [ ]:
#We now have a clean dataset
clean_df.head(10)

In [ ]:
clean_df.tail(10)

In [ ]:
#Check that we have all the CIDs for peace agreement
clean_df[clean_df['UN_agreement'] == 1].index
pa_UN_CID = pa.loc[pa['CID'].isin(clean_df[clean_df['UN_agreement'] == 1].index)]
pa_UN_CID['CID'].unique()

In [ ]:
#np.save('PA_Cleaned.csv' ,  pa_UN_CID)

In [ ]:
pa_UN_CID

In [ ]:
plt.figure(figsize=(20,10))
sns.countplot(x='UN', data= pa_UN_CID, hue='ended')

In [ ]:
plt.figure(figsize=(20,10))
sns.countplot(x='UN', data= pa, hue='ended')

In [ ]:
pa.groupby(['UN' , 'ended']).count()
# UN : 33% ended
#Not UN : 36% ended

In [ ]:
pa_UN_CID.groupby(['UN' , 'ended']).count()
#UN : 33% ended
#Not UN : 42% ended

## Data exploration

In this section, we will observe the distributions for the conflict over the `agreement` variable as we will want to have a propensity score matching for the conflicts that did or did not receive a peace agreement. (And later for conflicts that did or did not receive a peace aggreement from the UN).

We don't have any problems of missing values in this dataset, the only that we encounterd are for `enddate` and it simply means that the conflict is ongoing.

The correlations we are interrested in are the ones of the different features with the `agreement` variable. These will be checked when we will plot the distributions.

In [ ]:
#A function to have nice plots
def plot_distrib(s1, s2, title, xLabel, yLabel, ax=None):
    bins = np.histogram(s1)[1]
    sns.distplot(s1, kde=False, color=COLOR_NO_TREAT, norm_hist=True, ax=ax, bins=bins)
    sns.distplot(s2, kde=False, color=COLOR_TREAT, norm_hist=True, ax=ax, bins=bins)
    if ax is None:
        plt.title(title)
        plt.xlabel(xLabel)
        plt.ylabel(yLabel)
        plt.legend(['No agreement', 'Agreement'])

In [ ]:
#Distrib for the incomp variable
plt.figure(figsize=(12,5))
plot_distrib(s1=clean_df['incomp'][clean_df['agreement'] == 0], s2=clean_df['incomp'][clean_df['agreement'] == 1], title='incomp', xLabel = 'incomp', yLabel='Density', ax=None)

In [ ]:
#KS 2 sample test to check distrib
from scipy import stats 
print(('feature').ljust(10), ('statistic').ljust(10), ('p value').ljust(0), '\n') 
feature = 'incomp'
ks = stats.ks_2samp(clean_df['incomp'][clean_df['agreement'] == 0], clean_df['incomp'][clean_df['agreement'] == 1]) 
print(feature.ljust(10), '%.3f'.ljust(10) %ks[0], '%.3f'.ljust(0) %ks[1])

#### `incomp` distribution

A Kolmogorov–Smirnov 2 sample test will help us be sure of our analysis.
The Kolmogorov–Smirnov 2 sample test tests whether 2 samples are drawn from the same distribution. If the K-S statistic is small or the p-value is high, then we cannot reject the hypothesis that the distributions of the two samples are the same.

The `incomp` variable seems to be equally distributed across the `agreement` variable. Indeed, the plot and the Kolmogorov-Smirnov 2-sample tests both tell us that we can't reject the null hypothesis as the p-value is very high.

In [ ]:
#plot the distrib
plt.figure(figsize=(12,5))
plot_distrib(s1=clean_df['cumint'][clean_df['agreement'] == 0], s2=clean_df['cumint'][clean_df['agreement'] == 1], title='cumint', xLabel = 'cumint', yLabel='Density', ax=None)

In [ ]:
#KS 2 sample test to check distrib
print(('feature').ljust(10), ('statistic').ljust(10), ('p value').ljust(0), '\n') 
feature = 'cumint'
ks = stats.ks_2samp(clean_df['cumint'][clean_df['agreement'] == 0], clean_df['cumint'][clean_df['agreement'] == 1]) 
print(feature.ljust(10), '%.3f'.ljust(10) %ks[0], '%.3f'.ljust(0) %ks[1])

#### `cumint` distribution

Again the distributions are almost the same as seen on the plot. Furthermore, the p-value for the KS 2 sample test is very high (0.965) so we can't reject the null hypothesis.

In [ ]:
#plot the distrib
plt.figure(figsize=(12,5))
plot_distrib(s1=clean_df['type'][clean_df['agreement'] == 0], s2=clean_df['type'][clean_df['agreement'] == 1], title='type', xLabel = 'type', yLabel='Density', ax=None)

In [ ]:
#KS 2 sample test to check distrib
print(('feature').ljust(10), ('statistic').ljust(10), ('p value').ljust(0), '\n') 
feature = 'type'
ks = stats.ks_2samp(clean_df['type'][clean_df['agreement'] == 0], clean_df['type'][clean_df['agreement'] == 1]) 
print(feature.ljust(10), '%.3f'.ljust(10) %ks[0], '%.3f'.ljust(0) %ks[1])

#### `type` distribution

Here the distrib look a bit different on the plot but the p-value for the KS 2 sample test tells us that we can't reject the null hypothesis.

In [ ]:
#Date time so we can substract and have the duration
clean_df['startdate'] = pd.to_datetime(clean_df['startdate'])
clean_df['enddate'] = pd.to_datetime(clean_df['enddate'])
clean_df['duration'] = (clean_df['enddate'] - clean_df['startdate']).dt.days
clean_df['duration'].fillna(0 , inplace = True)

In [ ]:
#plot the distrib
plt.figure(figsize=(12,5))
plot_distrib(s1=clean_df['duration'][clean_df['agreement'] == 0], s2=clean_df['duration'][clean_df['agreement'] == 1], title='duration', xLabel = 'duration', yLabel='Density', ax=None)

In [ ]:
#KS 2 sample test to check distrib
print(('feature').ljust(10), ('statistic').ljust(10), ('p value').ljust(0), '\n') 
feature = 'duration'
ks = stats.ks_2samp(clean_df['duration'][clean_df['agreement'] == 0], clean_df['duration'][clean_df['agreement'] == 1]) 
print(feature.ljust(10), '%.3f'.ljust(10) %ks[0], '%.3f'.ljust(0) %ks[1])

#### `duration` distribution

As we have the date, we can know the duration of the conflict by substracting the `enddate` and the `startdate`. The duration is given in days. 

The distribution kind of look the same across the conflicts that did or did not receive a peace agreement.

In [ ]:
#Here we have to do the histogram ourselves
locationNAcount = clean_df[clean_df['agreement'] == 0].groupby('location').count()['sidea']
#Check the % for each location
locationNAcount = locationNAcount / len(clean_df[clean_df['agreement'] == 0])
locationNAcount.sort_values(ascending = False , inplace = True)
plt.figure(figsize=(12,5))
plt.title('Distribution of locations for conflicts without an agreement')
locationNAcount[:20].plot('bar')

In [ ]:
locationcount = clean_df[clean_df['agreement'] == 1].groupby('location').count()['sidea']
#Check the % for each location
locationcount = locationcount / len(clean_df[clean_df['agreement'] == 1])
locationcount.sort_values(ascending = False , inplace = True)
plt.figure(figsize=(12,5))
plt.title('Distribution of locations for conflicts with an agreement')
locationcount[:20].plot('bar')

#### `location` distribution

For the `location` variable the distribution looks quite different. On the other hand, this information is so different for every CID that it will be hard to use in our propensity score matching. We will prefer to check the distribution across the regions instead.

In [ ]:
regionNAcount = clean_df[clean_df['agreement'] == 0].groupby('region').count()['sidea']
#Check the % for each region
regionNAcount = regionNAcount / len(clean_df[clean_df['agreement'] == 0])
regionNAcount.sort_values(ascending = False , inplace = True)
plt.figure(figsize=(12,5))
plt.title('Distribution of regions for conflicts without an agreement')
regionNAcount[:5].plot('bar')

In [ ]:
regioncount = clean_df[clean_df['agreement'] == 1].groupby('region').count()['sidea']
#Check the % for each region
regioncount = regioncount / len(clean_df[clean_df['agreement'] == 1])
regioncount.sort_values(ascending = False , inplace = True)
plt.figure(figsize=(12,5))
regioncount.plot('bar')

#### `region` distribution

For the `region` variable the distribution looks very different. This will have to be investigated and we will maybe force this variable to be equal in our propensity score matching.

# Link with the desegregated DataSet

In [ ]:
precisedf = pd.read_csv('data/ged171.csv')

In [ ]:
precisedf.head(20)
#precisedf[precisedf['conflict_new_id'] == 205]

In [ ]:
precisedf.shape

In [ ]:
precisedf[precisedf['country']=='France']

In [ ]:
print('Number of unique conflicts in this dataset : %s' %(len(precisedf.conflict_new_id.unique())))

In [ ]:
filtereddf = precisedf.loc[precisedf['conflict_new_id'].isin(clean_df.index)]

In [ ]:
print('Number of unique conflicts in this dataset : %s' %(len(filtereddf.conflict_new_id.unique())))

In [ ]:
aggregatedf = filtereddf.groupby('conflict_new_id').sum()
aggregatedf['best']

In [ ]:
precisedf = precisedf[['conflict_new_id' , 'conflict_name' , 'best']]

In [ ]:
precisedfagg = precisedf.groupby('conflict_new_id').sum()

In [ ]:
clean_df

In [ ]:
precisedfagg.merge(right = clean_df, how = 'outer' ,  left_index= True , right_index = True)

In [ ]:
clean_df = clean_df.merge(right = precisedfagg , how = 'left' ,  left_index= True , right_index = True )

In [ ]:
#deadliest conflict
clean_df['best'].fillna(0 , inplace = True)
clean_df.sort_values(by = 'best' ,ascending=False)

# Propensity score matching between conflicts (UN as treatment)

In [ ]:
import sklearn.linear_model
import sklearn.preprocessing
#removing the useless features
ag = clean_df[clean_df['agreement'] == 1]
reg = ag[['incomp' , 'cumint' , 'type' , 'region' , 'duration']]
#prepocess by standardizing the dataset
reg = preprocessing.scale(reg)
#Perform a logistic regression in order to get the propensity score for each individuals
model = sklearn.linear_model.LogisticRegression()
model.fit(reg, ag.UN_agreement)
pred = model.predict_proba(reg)

In [ ]:
#Check the performance of the model
sum(model.predict(reg) == ag.UN_agreement)/len(reg)

In [ ]:
ag['CID'] = ag.index
ag

In [ ]:
#Scatter plot of the propensity score
plt.figure(figsize=(15, 10))
ag['pred'] = pred[:,1]
ax = sns.stripplot(x='CID', y='pred', hue='UN_agreement', data=ag, palette={0:"#e74c3c", 1: "#2ecc71"})
ax.set(xticklabels=[], ylabel='Treatment (prediction)')
plt.title('Propensity score per agreement')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., handles=ax.get_legend_handles_labels()[0], labels=['Without', 'With'], title='UN')

In [ ]:
import networkx as nx
from networkx.algorithms import bipartite

#create the nodes
G=nx.Graph()
G.add_nodes_from(ag['CID'][ag.UN_agreement == 0])
G.add_nodes_from(ag['CID'][ag.UN_agreement == 1])

In [ ]:
#Bipartite complete graph
for ID_u, score_u in zip(ag.CID[ag.UN_agreement == 0], ag.pred[ag.UN_agreement == 0]):
    for ID_v, score_v in zip(ag.CID[ag.UN_agreement == 1], ag.pred[ag.UN_agreement == 1]):
        G.add_edge(ID_u, ID_v, weight=-abs(score_u-score_v))

In [ ]:
print(G.number_of_nodes())
print(G.number_of_edges())

In [ ]:
#perform the matching
from networkx.algorithms import max_weight_matching
matching = max_weight_matching(G, maxcardinality=True)

In [ ]:
#We have 68 PAIDs that need to be matched, here we notice the dictionnary goes in both ways
len(list(matching.items()))

In [ ]:
ag_UN = ag[ag['UN_agreement'] == 1]
ag_UN['temp'] = 1
ag_UN = ag_UN[['CID' , 'pred' , 'temp']]
ag_nUN = ag[ag['UN_agreement'] == 0]
ag_nUN['temp'] = 1
ag_nUN = ag_nUN[['CID' , 'pred' , 'temp']]


#We merge the two datasets thanks to the temp column, this new merged dataset mimics the complete bipartite graph:
result = pd.merge(ag_nUN ,ag_UN , on='temp', how='left')
result = result[['CID_x' , 'CID_y' , 'pred_x' , 'pred_y']]
result['diff'] = abs(result['pred_x'] - result['pred_y'])
result = result.set_index(['CID_x', 'CID_y'])
result['diff'].isnull().values.any()
#result
print(result.loc[list(matching.items())].dropna().shape)
matched_CIDs = result.loc[list(matching.items())].dropna()
matched_CIDs

In [ ]:
matched_CIDs.index.levels[1]

In [ ]:
ag

In [ ]:
#A function to have nice plots
def plot_distrib(s1, s2, title, xLabel, yLabel, ax=None):
    bins = np.histogram(s1)[1]
    sns.distplot(s1, kde=False, color=COLOR_NO_TREAT, norm_hist=True, ax=ax, bins=bins)
    sns.distplot(s2, kde=False, color=COLOR_TREAT, norm_hist=True, ax=ax, bins=bins)
    if ax is None:
        plt.title(title)
        plt.xlabel(xLabel)
        plt.ylabel(yLabel)
        plt.legend(['UN agreement', 'Other Agreement'])

In [ ]:
#Distrib for the incomp variable
plt.figure(figsize=(12,5))
plot_distrib(s1=ag['incomp'][ag['UN_agreement'] == 1], s2=ag['incomp'][ag['Other_agreement'] == 1], title='incomp', xLabel = 'incomp', yLabel='Density', ax=None)

In [ ]:
#KS 2 sample test to check distrib
from scipy import stats 
print(('feature').ljust(10), ('statistic').ljust(10), ('p value').ljust(0), '\n') 
feature = 'incomp'
ks = stats.ks_2samp(ag['incomp'][ag['UN_agreement'] == 1], ag['incomp'][ag['Other_agreement'] == 1]) 
print(feature.ljust(10), '%.3f'.ljust(10) %ks[0], '%.3f'.ljust(0) %ks[1])

In [ ]:
#Distrib for the cumint variable
plt.figure(figsize=(12,5))
plot_distrib(s1=ag['cumint'][clean_df['UN_agreement'] == 1], s2=ag['cumint'][clean_df['Other_agreement'] == 1], title='cumint', xLabel = 'cumint', yLabel='Density', ax=None)

In [ ]:
#KS 2 sample test to check distrib
print(('feature').ljust(10), ('statistic').ljust(10), ('p value').ljust(0), '\n') 
feature = 'incomp'
ks = stats.ks_2samp(clean_df['cumint'][clean_df['UN_agreement'] == 1], clean_df['cumint'][clean_df['Other_agreement'] == 1]) 
print(feature.ljust(10), '%.3f'.ljust(10) %ks[0], '%.3f'.ljust(0) %ks[1])

In [ ]:
#Distrib for the type variable
plt.figure(figsize=(12,5))
plot_distrib(s1=ag['type'][clean_df['UN_agreement'] == 1], s2=ag['type'][clean_df['Other_agreement'] == 1], title='type', xLabel = 'type', yLabel='Density', ax=None)

In [ ]:
#KS 2 sample test to check distrib
print(('feature').ljust(10), ('statistic').ljust(10), ('p value').ljust(0), '\n') 
feature = 'incomp'
ks = stats.ks_2samp(clean_df['type'][clean_df['UN_agreement'] == 1], clean_df['type'][clean_df['Other_agreement'] == 1]) 
print(feature.ljust(10), '%.3f'.ljust(10) %ks[0], '%.3f'.ljust(0) %ks[1])

In [ ]:
#Distrib for the type variable
plt.figure(figsize=(12,5))
plot_distrib(s1=ag['duration'][clean_df['UN_agreement'] == 1], s2=ag['duration'][clean_df['Other_agreement'] == 1], title='duration', xLabel = 'duration', yLabel='Density', ax=None)

In [ ]:
#KS 2 sample test to check distrib
print(('feature').ljust(10), ('statistic').ljust(10), ('p value').ljust(0), '\n') 
feature = 'incomp'
ks = stats.ks_2samp(clean_df['duration'][clean_df['UN_agreement'] == 1], clean_df['duration'][clean_df['Other_agreement'] == 1]) 
print(feature.ljust(10), '%.3f'.ljust(10) %ks[0], '%.3f'.ljust(0) %ks[1])

In [ ]:
#A function to have nice plots
def plot_distrib(s1, s2, title, xLabel, yLabel, ax=None):
    bins = np.histogram(s1)[1]
    sns.distplot(s1, kde=False, color=COLOR_NO_TREAT, norm_hist=True, ax=ax, bins=bins)
    sns.distplot(s2, kde=False, color=COLOR_TREAT, norm_hist=True, ax=ax, bins=bins)
    if ax is None:
        plt.title(title)
        plt.xlabel(xLabel)
        plt.ylabel(yLabel)
        plt.legend(['Conflict solved by another entity than the UN', 'Conflict solved by the UN'])

In [ ]:
#Distrib for the incomp variable
plt.figure(figsize=(12,5))
plot_distrib(s1=ag['incomp'][ag['Other_agreement'] == 1][ag['UN_agreement']==0], s2=ag['incomp'][ag['UN_agreement']==1], title='incomp', xLabel = 'incomp', yLabel='Density', ax=None)

In [ ]:
print(('feature').ljust(10), ('statistic').ljust(10), ('p value').ljust(0), '\n') 
feature = 'incomp'
ks = stats.ks_2samp(ag['incomp'][ag['Other_agreement'] == 1][ag['UN_agreement']==0], ag['incomp'][ag['UN_agreement']==1]) 
print(feature.ljust(10), '%.3f'.ljust(10) %ks[0], '%.3f'.ljust(0) %ks[1])

In [ ]:
#Distrib for the incomp variable
plt.figure(figsize=(12,5))
plot_distrib(s1=ag['type'][ag['Other_agreement'] == 1][ag['UN_agreement']==0], s2=ag['type'][ag['UN_agreement']==1], title='type', xLabel = 'type', yLabel='Density', ax=None)

In [ ]:
print(('feature').ljust(10), ('statistic').ljust(10), ('p value').ljust(0), '\n') 
feature = 'type'
ks = stats.ks_2samp(ag['type'][ag['Other_agreement'] == 1][ag['UN_agreement']==0], ag['type'][ag['UN_agreement']==1]) 
print(feature.ljust(10), '%.3f'.ljust(10) %ks[0], '%.3f'.ljust(0) %ks[1])

In [ ]:
#Distrib for the incomp variable
plt.figure(figsize=(12,5))
plot_distrib(s1=ag['cumint'][ag['Other_agreement'] == 1][ag['UN_agreement']==0], s2=ag['cumint'][ag['UN_agreement']==1], title='cumint', xLabel = 'cumint', yLabel='Density', ax=None)

In [ ]:
print(('feature').ljust(10), ('statistic').ljust(10), ('p value').ljust(0), '\n') 
feature = 'cumint'
ks = stats.ks_2samp(ag['cumint'][ag['Other_agreement'] == 1][ag['UN_agreement']==0], ag['cumint'][ag['UN_agreement']==1]) 
print(feature.ljust(10), '%.3f'.ljust(10) %ks[0], '%.3f'.ljust(0) %ks[1])

In [ ]:
#Distrib for the incomp variable
plt.figure(figsize=(12,5))
plot_distrib(s1=ag['region'][ag['Other_agreement'] == 1][ag['UN_agreement']==0].astype(int), s2=ag['region'][ag['UN_agreement']==1].astype(int), title='region', xLabel = 'region', yLabel='Density', ax=None)

In [ ]:
print(('feature').ljust(10), ('statistic').ljust(10), ('p value').ljust(0), '\n') 
feature = 'region'
ks = stats.ks_2samp(ag['region'][ag['Other_agreement'] == 1][ag['UN_agreement']==0].astype(int), ag['region'][ag['UN_agreement']==1].astype(int)) 
print(feature.ljust(10), '%.3f'.ljust(10) %ks[0], '%.3f'.ljust(0) %ks[1])

In [ ]:
#Distrib for the incomp variable
plt.figure(figsize=(12,5))
plot_distrib(s1=ag['duration'][ag['Other_agreement'] == 1][ag['UN_agreement']==0].astype(int), s2=ag['duration'][ag['UN_agreement']==1].astype(int), title='region', xLabel = 'region', yLabel='Density', ax=None)

In [ ]:
print(('feature').ljust(10), ('statistic').ljust(10), ('p value').ljust(0), '\n') 
feature = 'duration'
ks = stats.ks_2samp(ag['duration'][ag['Other_agreement'] == 1][ag['UN_agreement']==0], ag['duration'][ag['UN_agreement']==1]) 
print(feature.ljust(10), '%.3f'.ljust(10) %ks[0], '%.3f'.ljust(0) %ks[1])

In [ ]:
ag.head()

In [ ]:
pa = pd.read_excel('data/ucdp-peace-agreements.xls')
searchfor = ['UN', 'United Nations' , 'united nations' , 'ONU']
pa_UN = pa[pa['pa_3rd'].astype(str).str.contains('|'.join(searchfor))]
inter = list(set(pa_UN.PAID).intersection(pa.PAID))
pa["UN"] = [1 if ele in inter else 0 for ele in pa.PAID]
pa[pa['UN'] == 1]

reader = csv.reader(open('data/translate_conf.csv', 'r'))

#Create a dictionnary that will have {New ID : Old ID} so we can make the link
d = {}
for row in reader:
    k, v = row
    d[v] = k
    
pa.CID = pa.CID.astype(str)
pa.CID = pa.CID.apply(lambda x: d[x])
pa.CID = pa.CID.astype(dtype = int)
pa.shape

In [ ]:
inter = list(set(pa.CID).intersection(matched_CIDs.index.levels[1]))
pa["matched"] = [1 if ele in inter else 0 for ele in pa.CID]
pa[pa['matched'] == 0]

# Inspect the .replace issue (SOLVED)

In [ ]:
#Import the Peace Agreement dataset and the translation table to get the new IDs
pa = pd.read_excel('data/ucdp-peace-agreements.xls')
searchfor = ['UN', 'United Nations' , 'united nations' , 'ONU']
pa_UN = pa[pa['pa_3rd'].astype(str).str.contains('|'.join(searchfor))]
inter = list(set(pa_UN.PAID).intersection(pa.PAID))
pa["UN"] = [1 if ele in inter else 0 for ele in pa.PAID]
pa[pa['UN'] == 1]

reader = csv.reader(open('data/translate_conf.csv', 'r'))

#Create a dictionnary that will have {New ID : Old ID} so we can make the link
d = {}

for row in reader:
    k, v = row
    d[v] = k
    
pa.CID = pa.CID.astype(str)
#print(type(pa.CID))
pa.CID = pa.CID.apply(lambda x: d[x])
#pa = pa.replace(to_replace = d , )
#pa.CID = pa.CID.astype(dtype = int)

In [ ]:
df = pd.read_csv('data/ucdp-prio-acd-171.csv')
df.drop(['sidea2nd', 'sideb2nd' , 'terr' , 'year' , 'intensity' , 'startprec' , 'startdate2' , 'epend' , 'ependprec', 'gwnoa', 'gwnoa2nd', 'gwnob', 'gwnob2nd', 'gwnoloc', 'startprec2', 'version', 'sidebid'], axis=1, inplace = True)
clean_df = df.groupby('conflictid').last()
clean_df.head()

In [ ]:
#pa[pa['PAID'] == 1001999050601]
clean_df.loc[335]

In [ ]:
pa = pd.read_excel('data/ucdp-peace-agreements.xls')
pa[pa['PAID'] == 1001999050601]

# For the next markdown :

## Propensity score matching agreements / no agreements :

Even if the distributions across the variables looks quite the same, it will be interresting to check how a propensity score matching works on this data and then check if a peace agreement helps to bring stability in a region or not. We will have to code the later variable. We already have a few ideas on how to do that.

## Propensity score matching agreements / UN agreements : 

Then we will run the same analysis as we previously did but this time we will filter the CIDs to be distributed the same before doing another propensity score matching on the agreements so we are sure our analysis is not naive.

## Plot the map of the conflicts/ Peace Agreements

### Attention je crois que des pays s'affichent en couleur alors que pas de conflit (Pb de cut de date je pense)

### Est-ce qu'on met des couleurs à des endroits ou il n'y a pas de conflit ?(si oui je pense qu'on ne devrait pas)

### Dès qu'on clique les couleurs apparaissent

### Check le "conflit de 2001 à 2016" aux states, ca semble bizarre

In [ ]:
clean_df[clean_df['location'] == 'United States of America']

In [ ]:
df = pd.read_csv('data/ucdp-prio-acd-171.csv')
df[][df['conflictid'] == 418]

## Map data cleaning
The goal here is to clean the geographical data columns so that the countries' names match with the names in the geojson file.

In [ ]:
data = json.load(open('./data/countries.geo.json'))
map_countries = set([country['properties']['name'] for country in data['features']])

In [ ]:
# takes a string as input and returns the cleaned string
def clean_location(location):
    location = re.sub(r'\([^)]*\)', '', location)
    location = re.sub(', ', ',', location)
    location = re.sub(' ,', ',', location)
    return location.strip()

In [ ]:
clean_df.location = clean_df.location.apply(clean_location)

In [ ]:
# let's check how many countries from the data still have no matcing on the map
def print_unmatched(map_countries):
    data_countries = set(clean_df.location.values)
    no_match = []
    for entry in data_countries:
        # some entries have several countries separated by a ','
        for country in entry.split(','):
            if country not in map_countries:
                no_match.append(country)
    print("There are " + str(len(set(no_match))) + " locations that are not on the map:")
    print(set(no_match))
print_unmatched(map_countries)

Let's do some manual corrections for the few locations that are still unmatched:

In [ ]:
manual_matchings = {
    'Bosnia-Herzegovina' : 'Bosnia and Herzegovina',
    'Tanzania' : 'United Republic of Tanzania',
    'Comoros' : 'Madagascar',
    'South Yemen' : 'Yemen',
    'DR Congo' : 'Democratic Republic of the Congo',
    'Hyderabad' : 'India',
    'South Vietnam' : 'Vietnam',
    'FYR' : 'Macedonia',
    'Grenada' : 'Spain',
    'Rumania' : 'Romania',
    'Serbia' : 'Republic of Serbia',
    'Congo' : 'Republic of the Congo',
    'Guinea-Bissau' : 'Guinea Bissau',
}
def replace_names(location):
    names = list(set([manual_matchings[x] if x in manual_matchings else x for x in location.split(',')]))
    return ','.join(names)
clean_df.location = clean_df.location.apply(replace_names)

In [ ]:
print_unmatched(map_countries)

### Add the locations' IDs from the GeoJson to the dataframe
The GeoJSON contains for each location a unique ID. We add these locations to the dataframe to use them as unique identifiers in the front-end.

In [ ]:
# build location -> locID dictionnary
loc_id_dict = dict()
for country in data['features']:
    loc_id_dict[country['properties']['name']] = country['id']
    if country['id'] == '-99':
        print(country['properties']['name'])
        
def getLocationIds(locations):
    ids = []
    for loc in locations.split(','):
        ids.append(loc_id_dict[loc])
    return ','.join(ids)
        
# Add locationID to the dataframe
clean_df['locationID'] = clean_df.location.apply(getLocationIds)
clean_df.head(30)

In [ ]:
# Add location IDs to the dataframe
#clean_df.head()
#clean_df['locationID'] = clean_df.location.map(loc_id_dict)
print(clean_df.location.unique().size)
print(clean_df['locationID'].unique().size)
#clean_df.head()

### Export the data as a csv file
Now the data is clean, let's export it in a csv file to use it in the front-end.

In [ ]:
clean_df.to_csv('conflicts.csv', sep='\t')

In [ ]:
clean_df[clean_df.location == 'Russia']

In [ ]:
clean_df.enddate.max()